In [2]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import xarray as xr

% matplotlib inline

### UN Population Data Analysis

**Data & Scope:** downloaded on 12/12 3.40 pm
- 1950-2100
    - historical/current: 1950-2015 are estimates
    - projections (probabilistic): 2016 and beyond (9 different variants)
- Population data only.
- Update status: 
    - last update - 20 Aug 2013 
    - next udpate - summer 2019
- Refer to *Look at Data* Section below for detailed information on variables
in data

**Source:** United Nations (http://data.un.org/Data.aspx?d=PopDiv&f=variableID%3a12)
- total population - `WPP2017_TotalPopulationBySex.csv`
- meta data (233/ countries only) `WPP2017_F02_METAINFO.csv` 
- meta data (273/ countries, regions and groupings) `WPP2017_F01_LOCATIONS.csv`

**Analysis Goals**
- Compare UN population data with PWT, WB, and IMF data
- Outline UN's methodology for population data, note any limitations/assumptions.

*Currently Out of Scope*
- Also evaluate whether UN dataset seems reliable for future projection X 

**Conclusion**

**Questions**


### Table of contents

1. Open data and metadata
2. Look at data. Columns. Define variables. Shape. # unique values. Any groups?
3. Filter variant (Medium only. For historical all variants have same population value. but picking Medium since it's most used). 
4. Filter by years (historical data only 1950-2015)
5. Merge data and meta data -> to filter only countries (i.e. exclude non-country groupings)
6. Select only certain columns (of interest)
7. Pivot data to reformat (make years columns, to be consistent with WB)
8. Reindex data to have an index column? (optional) 

#### 1. Open UN data & metadata

In [3]:
root_dir = os.environ['ZERG']
#os.getcwd()

In [4]:
df = pd.read_csv(root_dir + 'data/UN/WPP2017_TotalPopulationBySex.csv')
df_meta = pd.read_excel(root_dir + 'data/UN/WPP2017_F02_METAINFO_mod.xlsx', sheetname='country_meta_subset', skip_rows=13)

In [5]:
df_meta.head(2)

,Index,Country or area,Notes,Country code,ISO3
0,1,Afghanistan,NaN,4,AFG
1,2,Albania,NaN,8,ALB


#### Open Other Data Sources (WB, PWT)

In [6]:
df_wb = pd.read_csv('/Users/mlimb/Dropbox/covariate_project/zerg/data/WB/population/API_SP.POP.TOTL_DS2_en_csv_v2.csv', skiprows=3)
df_pwt = pd.read_stata('/Users/mlimb/Dropbox/covariate_project/zerg/data/PWT/pwt90.dta')

#df_pwt.head(5)

#### 2. Look at UN data

Variables/columns:
`LocID` (int) integer code associated with location. # choice of numbers seem alphabetical

    (Development group, Income group, Region-eg. Africa, Subregion-eg. Eastern Africa)

`Location` (Str) country / region name

`VarID` (int) ID associated with one of the 9 variants (used for future projection). 2 - Medium.

`Variant` (Str) variant type. One of ['Medium', 'High', 'Low', 'Constant fertility',
       'Instant replacement', 'Zero migration', 'Constant mortality',
       'No change', 'Momentum']

`Time` (int) year, ranging from 1950 to 2100

`MidPeriod` (float) mid-year as given by 1950.5... 2100.5

`PopMale` (float) total male population

`PopFemale` (float) total female population

`PopTotal` (float) total population, both sexes


In [7]:
df.columns

Index(['LocID', 'Location', 'VarID', 'Variant', 'Time', 'MidPeriod', 'PopMale',
       'PopFemale', 'PopTotal'],
      dtype='object')

In [8]:
df.shape

(371007, 9)

In [9]:
df.columns

Index(['LocID', 'Location', 'VarID', 'Variant', 'Time', 'MidPeriod', 'PopMale',
       'PopFemale', 'PopTotal'],
      dtype='object')

In [10]:
print(df['Time'].nunique()) # 151 years
print(df['Location'].nunique()) # 273 unique country/areas
print(df['Variant'].nunique())# 9 variants # 

151
273
9


In [11]:
df['Time'].unique()
df['Time'].unique().min(), df['Time'].unique().max() # get year range

(1950, 2100)

In [12]:
# trying groups to filter country/non-country (eg region)

In [13]:
# g = df.groupby('Location')
# g.groups.get('Africa')
# #g.get_group('Western Africa')# Note: groups include regions too 
# g.get_group('World') # Note: groups include World too
# g.get_group('Less developed regions') # groups include development group too... then likely income group too
# g.get_group('Low-income countries') # groups include incomegroup too

# # need to exclude World, Development, Income groups, etc.
# # get filter via LocID**

In [14]:
df['Variant'].unique()

array(['Medium', 'High', 'Low', 'Constant fertility',
       'Instant replacement', 'Zero migration', 'Constant mortality',
       'No change', 'Momentum'], dtype=object)

### 3. Filter data ('medium' variant)  - only relevant to some datasets from UN that have multiple variants
Note: for historical data, all variants have same population value. For future projection, medium is the most used variant (does not imply highest accuracy) as population projections are inherently very uncertainy and depends on many variables.

In [15]:
# Filter medium variant
df_med = df[df['Variant']=='Medium']

In [16]:
df_med.columns
df_med.shape

(41223, 9)

### 4. Filter historical estimates only (Do this again after merge -- checking)

In [17]:
print(len(df_med['Time'].unique()) == (2100- 1950+1)) 
df_med['Time'].unique().max(), df_med['Time'].unique().min()

True


(2100, 1950)

In [18]:
df_hist = df_med[df_med['Time'] <= 2015]
df_hist['Time'].unique().min(), df_hist['Time'].unique().max() # now year ranges from 1950 to 2015

(1950, 2015)

In [19]:
df_hist.shape # (18018, 9) fewer entries due to reduction in years

(18018, 9)

In [20]:
df_hist.columns
df_hist.head(3)

,LocID,Location,VarID,Variant,Time,MidPeriod,PopMale,PopFemale,PopTotal
0,4,Afghanistan,2,Medium,1950,1950.5,4099.243,3652.874,7752.118
1,4,Afghanistan,2,Medium,1951,1951.5,4134.477,3705.031,7839.510
2,4,Afghanistan,2,Medium,1952,1952.5,4173.993,3760.979,7934.980


### 5. Merge meta + data to only retrieve country data (expect 233 entries)

###  Note: merge on `LocID` and `Country code` instead of country name. Three country names cause problems during merge, including:

**<span style="color:black; background-color: #FFEBCD">Warning</span>**

*  "C̫te d'Ivoire" vs "Côte d'Ivoire"
*  'Cura̤ao' vs 'Curaçao'
* ?

In [21]:
a = sorted(df_meta['Country code'].unique())
b = sorted(df['LocID'].unique())
set(a) - set(b)

# a == b

set()

In [22]:
print(df_hist.columns) # Location, # LocID
print(df_meta.columns) # Country or area # Country code?

# add new column to merge on. same data as 'Country or area' but too lazy to rename
df_meta['LocID'] = df_meta['Country code']

# alternatively. can rename `Country code` column

Index(['LocID', 'Location', 'VarID', 'Variant', 'Time', 'MidPeriod', 'PopMale',
       'PopFemale', 'PopTotal'],
      dtype='object')
Index(['Index', 'Country or area', 'Notes', 'Country code', 'ISO3'], dtype='object')


In [23]:
# similiar, but different from WB filter_non_countries:
# 1) requires a column name to merge on (as a parameter)
# 2) doesn't create as mask for non-country using meta-data (to exclude groupings)
# 2) is separated out as a different function

def merge_dfs_on(_df, _metadata, _merge_col):
    '''
    _df : pd.DataFrame
        either income or population data
    _metadata : pd.DataFrame
        metadata on a list of entries including countries and non-countries 
        data source is from the World Bank
        has IncomeGroup column that is not null for countries (217)
    _merge_col : Str
        column name to merge on
    '''
    _merged = _df.merge(_metadata, on=_merge_col)
    return _merged

merged = merge_dfs_on(df_hist, df_meta, 'LocID')
merged.shape

(15378, 14)

In [24]:
# Also from WB_analysis `get_row_by_country_name`

def get_country(_df, country_name):
    return _df[_df['Location'] == country_name]

def get_country_by_first_letter(df, first_letter):
    return df.loc[df['Location'].str.startswith(first_letter)]

### 6. Select interested columns

In [25]:
#Given dataframe and a list of cols, return subset containing given columns only
select_cols= lambda _df, _list_of_cols: _df[_list_of_cols]

list_of_cols_wanted = ['LocID', 'ISO3', 'Location', 'Time', 'PopTotal']
    
df_sub = select_cols(merged, list_of_cols_wanted)

In [26]:
df_sub.shape
df_sub['Time'].unique()

array([1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960,
       1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971,
       1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982,
       1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993,
       1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004,
       2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015])

### Pandas pivoting!!

In [27]:
get_country_by_first_letter(df_sub, 'I')['Location'].unique()

array(['Iceland', 'India', 'Indonesia', 'Iran (Islamic Republic of)',
       'Iraq', 'Ireland', 'Isle of Man', 'Israel', 'Italy'], dtype=object)

In [28]:
g = df_sub.groupby('Location')
len(g.groups.keys()) #233

# 233 countries

233

In [29]:
pivoted=df_sub.pivot_table(index='Location', columns='Time', values='PopTotal')
pivoted.shape

(233, 66)

In [30]:
pivoted.columns

Int64Index([1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960,
            1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971,
            1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982,
            1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993,
            1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004,
            2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015],
           dtype='int64', name='Time')

In [31]:
2015-1950+1 # years

66

### Reindexing to make Location (or country name) a separate column

Doing this to be consistent with WB datasets. Right now index column has name `Time` but it's irrelevant so not trying to rename it.

But `pivoted` and not `reindexed` has nice advantage of row access via country name.

In [32]:
df_un= pivoted.reset_index(inplace=False)

In [36]:
pivoted.to_csv('un_population_1215.csv')